<a href="https://colab.research.google.com/github/lonen8188/PythonAI/blob/5-2.scikit-learn/hg_05_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 교차 검증과 그리드 서치
# 검증세트 : 훈련세트(60%), 검증세트(20%), 테스트세트(20%)
# 테스트하고 싶은 매개변수를 바꿔가면서 가장 좋은 모델이 나올 수 있음
# 이 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련함
# 마지막 테스트에서 최종 점수를 평가함

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
# 훈련세트와 테스트 세트를 나눔
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42) # 20%

#검증세트 20% 생성
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)
# 원래 5197개였던 훈련세트가 4157개로 줄고, 검증세트가 1040개가 됨

(4157, 3) (1040, 3)


In [3]:
# sub_input, sub_target과 val_input, val_target을 사용해 모델을 만들고 평가함.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target) # 훈련

print(dt.score(sub_input, sub_target)) # 출력 0.9971133028626413(과대적합)
print(dt.score(val_input, val_target))
# 매개변수를 바꿔서 더 좋은 모델을 찾아야 함

0.9971133028626413
0.864423076923077


In [8]:
# 검증세트를 만드느냐고 훈련 세트가 줄었음. (많은 데이터를 훈련하면 좋은 결과가 나옴)
# 교차 검증 : 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함
# [훈련세트, 훈련세트, 검증세트]
# [훈련세트, 검증세트, 훈련세트]
# [검증세트, 훈련세트, 훈련세트] -> 패리티 방식(3-폴드 교차 검증)
# 5-폴드, 10-폴드도 있음
# 사이킷런에는 cross_validate()라는 교차 검증 함수를 사용

from sklearn.model_selection import cross_validate
# 평가할 모델 객체를 첫번째 매개변수로 전달함(직접 검증세트를 떼지않고 훈련세트 전체를 전달함)
scores = cross_validate(dt, train_input, train_target)
#scores = cross_validate(dt, train_input, train_target, cv=5) # cv 매개변수를 이용해 폴드 수를 변경함 cv=5(기본값)
print(scores)
# fit_time(훈련시간), score_time(검증시간), test_score(최종점수) 키를 가진 딕셔너리(k : v)를 반환함

{'fit_time': array([0.01027155, 0.00810814, 0.00937915, 0.00869703, 0.00803661]), 'score_time': array([0.00133681, 0.00128794, 0.00126815, 0.00140834, 0.00168252]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np

print(np.mean(scores['test_score'])) # test_score 이름이지만 검증폴드의 점수임

0.855300214703487


In [10]:
# 교차 검증을 수행하면 입력한 모델에서 얻을 수 잇는 최상의 검증 점수를 가늠해 볼 수 있다.
# 주의 : cross_validate()는 훈련세트를 섞어서 폴드를 나누지 않음 -> 분할기 사용(splitter)
# 사이킷런의 분할기는 교차검증에서 폴드를 어떻게 나눌지를 결정함 -> cross_validate()는 기본적으로 회귀모델인 KFold 분할기를 사용함

from sklearn.model_selection import StratifiedKFold
# https://continuous-development.tistory.com/entry/MLDL-python-%EC%9D%84-%ED%86%B5%ED%95%9C-%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D-k-Fold-stratifiedkFold#google_vignette

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [15]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 10-폴드 교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [17]:
# 결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봐야 함
# 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 골라야 함

# 하이퍼파라미터 튜닝
# 모델 파라미터 : 머신런닝 모델이 학습하는 파라미터
# 하이퍼파라미터 : 사용자가 지정해야만 하는 파라미터
# 하이퍼파라미터 튜닝 : 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
# 그 다음 검증세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 변경 1~2, 5~6개의 매개변수를 제공
# AutoML : 사람의 개입없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술

# Max_depth 를 최적으로 고정하고 min_sample_split을 바꿔가며 최적의 값을 찾는다.면 값이 함께 변경됨
# -> 두개의 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 함 -> 복잡해 짐
# 사이킷런에서 제공하는 그리드 서치(Grid Search)가 제공 GridSearchCV

from sklearn.model_selection import GridSearchCV
# min_impurity_decrease 매개변수의 최적값을 찾아보자
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # 0.0001~ 증가 5번

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 결정 트리 클래스의 객체를 생성하자마자 바로 전달 함
# 일반 모델을 훈련하는 것 처럼 fit() 메서드를 호출 함
# 이 메서드를 호출 하면 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가면서 총 5번 수행
# GridSearchCV의 cv 매개변수 기본 값은 5임 5*5 =25 번 수행
# n_jobs에서 병렬 실행에 사용될 CPU 코어수를 지정함 (-1로 사용시 모든 코어)
gs.fit(train_input, train_target) # 훈련시작
# 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만
# 사이킷런의 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련 함

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [18]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [19]:
print(gs.best_params_) # 그리드 서치가 찾은 최적을 값을 가진 변수

{'min_impurity_decrease': 0.0001}


In [20]:
print(gs.cv_results_['mean_test_score']) # 교차 검증으로 얻은 점수
# 첫번째 값이 가장 큼, 수동으로 고르는 것보다 argmax() 함수를 이용하면 가장 큰 값의 인덱스를 활용함

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [21]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # 0.86819297
print(gs.cv_results_['params'][best_index])
# 과정 요약
# 1. 탐색할 매개변수 지정
# 2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음 -> 그리드 서치에 저장
# 3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련 -> 그리드 서치에 저장

{'min_impurity_decrease': 0.0001}


In [22]:
# 조금 더 복잡한 매개변수 조합 해보겠음
# 노드를 분할하기 위한 불순도 감소 최소량 지정 max_depth(트리의 깊이)
# min_samples_split 노드를 나누기 위한 최소 샘플 수
# 넘파이 arange() 함수는 첫번째 매개변수 값에서 시작해서 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 도한 배열을 만듬
# 0.0001 ~ 0.001까지 0.00001을 더한 배열 (두번째 매개변수는 포함되지 않음) = 9번
# 파이썬 ragne() 함수는 정수만 사용가능 5~20까지 1씩 증가 = 15개
# 교차 검증 회수 9*15*10=1350 개 * 5-폴드교차 = 6750개
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target) # 훈련시작

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
print(gs.best_params_) # 최상의 매개변수 조합확인

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 점수 확인
# 개선점 : 탐색할 매개변수 간격을 0.00001 혹은 1로 설정 했는데 근거 부족(좁히거나 넓힐 필요성?)

0.8683865773302731


In [25]:
# 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음
# 너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래 걸림
# 해결 : 랜덤 서치를 사용해보자.

# 랜덤서치 : 매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 잇는 확률 분포도 객체를 전달
# 싸이파이 : 파이썬의 핵심 과학 라이브러리로 적분,보간, 선형대수, 확률 등 포함한 수치 계산용 전용 라이브러리(코랩에 내장)

from scipy.stats import uniform, randint
# uniform, randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.(균등 분포에서 샘플링)
# randint()에서는 정수값을 추출, uniform()은 실수값을 뽑음

In [26]:
rgen = randint(0, 10) # 10개 숫자를 샘플링(교재와 다름)
rgen.rvs(10)

array([8, 9, 0, 8, 5, 3, 6, 3, 6, 2])

In [27]:
np.unique(rgen.rvs(1000), return_counts=True) # 1000개를 샘플링해서 각 숫자의 개수를 세어봄

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([108,  97,  97,  98, 106,  89, 105,  88, 104, 108]))

In [28]:
ugen = uniform(0, 1) # 실수도 테스트
ugen.rvs(10)

array([0.0473583 , 0.72126337, 0.79926781, 0.4479327 , 0.24740343,
       0.82696135, 0.30106008, 0.93944084, 0.84339983, 0.99796453])

In [30]:
# 탐색할 매개변수의 딕서너리를 만들어 봄
# min_samples_leaf을 탐색 대상에 추가
params = {'min_impurity_decrease': uniform(0.0001, 0.001), # 0.00001 ~ 0.001의 실수값
          'max_depth': randint(20, 50),                   # 20~50 사이의 정수
          'min_samples_split': randint(2, 25),            # 2~25 사이의 정수
          'min_samples_leaf': randint(1, 25),             # 1~25 사이의 정수
          # 리프 노드가 되기 위한 최소 샘플 개수(지식 노드의 샘플 수가 이 값보다 작으면 분할 하지 않음)
          }

In [31]:
# 샘플링 회수는 사이킷런의 램덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
# n_iter=100 총 100번을 샘플링하여 교차 검증을 수행(최적의 매개변수 조합을 찾음)

gs.fit(train_input, train_target)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79cbd9229fc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79cbd9228eb0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79cbd92284c0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79cbd9228b80>},
                   random_state=42)

In [32]:
print(gs.best_params_) # 결과 최적의 매개변수 조합을 찾음

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최고의 교차 검증 점수 확인

0.8695428296438884


In [34]:
dt = gs.best_estimator_
# 최종 모델로 결정하고 테스트 세트의 성능을 확인
print(dt.score(test_input, test_target))

0.86


In [ ]:
# 테스트 세트 점수는 검증세트 점수보다 조금 작은 것이 일반적임
# 충분히 다양한 매개변수를 테스트해서 얻은 점수니 알고 진행 함.